In [3]:
import pandas as pd
import numpy as np
import os

# --- Configuration ---
# Input file path for your non-adjusted stock price history
input_file_path = 'company_data/C38U.SI_stock_price_history_non_adj.csv'

# Output directory for the enhanced data
output_directory = 'calculated_averages'
# Output file name for the enhanced data
output_file_name = 'C38U.SI_stock_price_history_with_indicators.csv'

# --- 1. Load the Data from your CSV file ---
print(f"Attempting to load data from: {input_file_path}")
try:
    df = pd.read_csv(input_file_path)
    print("Successfully loaded data.")
except FileNotFoundError:
    print(f"Error: The file at {input_file_path} was not found. Please check the path and filename.")
    exit() # Exit the script if the file isn't found
except Exception as e:
    print(f"An unexpected error occurred during file loading: {e}")
    exit()

# --- 2. Initial Data Preparation: Ensure Correct Data Types and Set Index ---
print("\nPerforming initial data preparation (Date conversion, setting index, numeric coercion)...")
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True) 
# Set 'Date' as the index and sort it, which is crucial for time-series operations
df = df.set_index('Date').sort_index()

# Ensure Open, High, Low, Close, and Volume columns are numeric.
# Using errors='coerce' will turn any non-convertible values into NaN (Not a Number).
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['High'] = pd.to_numeric(df['High'], errors='coerce')
df['Low'] = pd.to_numeric(df['Low'], errors='coerce')
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# Remove commas from the 'Volume' column first, then convert to numeric
df['Volume'] = df['Volume'].astype(str).str.replace(',', '', regex=False)
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

print("\n--- Initial Data Snapshot ---")
print(df.head())
print("\n--- Initial Data Info ---")
print(df.info())
print(f"Total rows loaded: {len(df)}")

# --- 3. Data Cleaning: Remove Non-Trading (e.g., Dividend/Split) Rows and Handle Duplicates ---

print("\n--- Starting Data Cleaning Process ---")
initial_row_count = len(df)

# Step 3.1: Identify and Remove Rows Likely Representing Non-Trading Events
# Common indicators for non-trading/dividend rows: Volume is 0 or NaN, or Close price is NaN.
print(f"\nChecking for non-trading rows (Volume 0/NaN or Close NaN)...")
non_trading_rows_mask = (df['Volume'] == 0) | (df['Volume'].isna()) | (df['Close'].isna())

num_non_trading_rows = non_trading_rows_mask.sum()
print(f"Found {num_non_trading_rows} rows likely representing non-trading events.")

# Keep only the rows that are NOT identified as non-trading events.
df = df[~non_trading_rows_mask].copy() # Use .copy() to ensure it's a new DataFrame and avoid warnings

rows_removed_events = initial_row_count - len(df)
if rows_removed_events > 0:
    print(f"Removed {rows_removed_events} non-trading event rows.")
else:
    print("No non-trading event rows found or removed.")

# Step 3.2: Handle Any Remaining Date Duplicates (after removing event rows)
print(f"\nChecking for remaining duplicate dates after non-trading event removal...")
rows_before_duplicate_check = len(df)

# Identify all rows where the index (Date) is duplicated, marking all occurrences.
duplicate_dates_mask = df.index.duplicated(keep=False)
num_duplicate_dates = duplicate_dates_mask.sum()
print(f"Found {num_duplicate_dates} rows with duplicate dates (counting all instances) for removal.")

# Filter the DataFrame to keep only non-duplicate index entries, keeping the first occurrence.
df = df[~df.index.duplicated(keep='first')]

rows_removed_duplicates = rows_before_duplicate_check - len(df)
if rows_removed_duplicates > 0:
    print(f"Removed {rows_removed_duplicates} remaining duplicate date entries.")
else:
    print("No remaining duplicate dates found or removed.")

# Final check for any critical NaNs that might have been missed or introduced.
# This ensures that essential columns for calculations are clean.
df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)
print(f"\nTotal rows after all initial cleaning steps: {len(df)}")

print("\n--- Data after Cleaning Snapshot ---")
print(df.head())
print("\n--- Data Info after Cleaning ---")
print(df.info())


# --- 4. Calculate Technical Indicators ---
print("\n--- Calculating Technical Indicators ---")

# 4.1. Moving Averages
df['50-MA'] = df['Close'].rolling(window=50).mean()
df['200-MA'] = df['Close'].rolling(window=200).mean()
print("Calculated 50-day and 200-day Moving Averages.")

# 4.2. Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    # Handle cases where loss is zero to avoid division by zero
    rs = np.where(loss == 0, np.inf, gain / loss)
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df, window=14)
print("Calculated Relative Strength Index (RSI).")

# 4.3. MACD (Moving Average Convergence Divergence)
# Helper function for Exponential Moving Average
def calculate_ema(data_series, span):
    return data_series.ewm(span=span, adjust=False).mean()

# Calculate 12-period EMA and 26-period EMA
exp1 = calculate_ema(df['Close'], span=12)
exp2 = calculate_ema(df['Close'], span=26)

# MACD Line
df['MACD'] = exp1 - exp2
# Signal Line (9-period EMA of MACD Line)
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
# MACD Histogram
df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
print("Calculated MACD, MACD Signal, and MACD Histogram.")

# 4.4. Volatility (Standard Deviation of Returns)
df['Daily_Return'] = df['Close'].pct_change()
df['Volatility'] = df['Daily_Return'].rolling(window=20).std() # 20-day rolling standard deviation
print("Calculated Volatility (Standard Deviation of Returns).")

# --- 5. Final Cleaning: Handle NaN values introduced by indicator calculations ---
print("\n--- Final Cleaning: Handling NaNs from indicator calculations ---")
rows_before_final_dropna = len(df)

# List of all columns that should ideally not have NaNs for model training
columns_to_check_for_nan = [
    'Open', 'High', 'Low', 'Close', 'Volume', # Re-check essential price data
    '50-MA', '200-MA', 'RSI', 'MACD', 'MACD_Signal', 'MACD_Histogram', 'Volatility', 'Daily_Return'
]
# Filter the list to include only columns actually present in the DataFrame
columns_to_check_for_nan = [col for col in columns_to_check_for_nan if col in df.columns]

# Drop rows where any of the critical columns have NaN values
df_cleaned = df.dropna(subset=columns_to_check_for_nan)

num_rows_dropped_indicators = rows_before_final_dropna - len(df_cleaned)
if num_rows_dropped_indicators > 0:
    print(f"Removed {num_rows_dropped_indicators} rows due to NaNs from indicator calculations (e.g., start of series).")
else:
    print("No additional rows dropped due to indicator NaNs.")

print(f"Final number of rows in the cleaned DataFrame: {len(df_cleaned)}")

print("\n--- Final Cleaned Data Snapshot (first 5 rows) ---")
print(df_cleaned.head())
print("\n--- Final Cleaned Data Info ---")
print(df_cleaned.info())
print("\n--- Final Cleaned Data (last 5 rows) ---")
print(df_cleaned.tail())

# --- 6. Write the newly generated DataFrame to the specified folder ---
output_full_path = os.path.join(output_directory, output_file_name)

print(f"\nAttempting to save enhanced data to: {output_full_path}")
# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

try:
    # Save the DataFrame to CSV. index=True ensures the 'Date' index is saved as a column.
    df_cleaned.to_csv(output_full_path, index=True)
    print("Enhanced data successfully saved!")
except Exception as e:
    print(f"Error saving the data to CSV: {e}")

print("\nScript execution complete.")

Attempting to load data from: company_data/C38U.SI_stock_price_history_non_adj.csv
Successfully loaded data.

Performing initial data preparation (Date conversion, setting index, numeric coercion)...

--- Initial Data Snapshot ---
            Open  High   Low  Close     Adj      Volume
Date                                                   
2019-12-31  2.43  2.47  2.43   2.46  2.0321   4710400.0
2020-01-02  2.44  2.47  2.43   2.46  2.0321   8248900.0
2020-01-03  2.46  2.47  2.43   2.45  2.0239   5668600.0
2020-01-06  2.44  2.45  2.42   2.43  2.0074  10843500.0
2020-01-07  2.44  2.47  2.43   2.46  2.0321  12045600.0

--- Initial Data Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1268 entries, 2019-12-31 to 2024-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1257 non-null   float64
 1   High    1257 non-null   float64
 2   Low     1257 non-null   float64
 3   Close   1257 non-null   float64
 4  

Added the below on 18 June to include Simple Moving Average and Exponential Moving Average calculation